<img src=../images/gdd-logo.png width=300px align=right>

# Unsupervised Learning

**Supervised learning** algorithms are machine learning algorithms that learn with the help of external feedback: the algorithm makes a prediction, compares its prediction to a provided ground truth, and "learns" by adjusting its internal parameters. 

In contrast, the techniques described in this lecture do not rely on some external notion of what is or is not correct; this class of learning techniques is referred to as **unsupervised learning**. 

For unsupervised learning, we can roughly differentiate between two categories: 
   - Clustering
   - Dimensionality reduction

In this notebook, we will take a look at the different families of clustering algorithms: partional, density-based and hierarchical clustering.

# Clustering families

- [Partitional clustering]()
- [Density-based clustering]()
- [Hierarchical clustering]()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Partitional clustering

### K-means
K-means is a widely used clustering algorithm that assigns each point in the dataset to a cluster. 

1. A number (_k_) of _centroids_ are initialised. These are the centers of our clusters. Usually, these centroids are data points in the data set. 
2. Each point in the dataset gets assigned to one out of _k_ clusters based on the minimal Euclidean distance between the data point and each centroid. 
3. The centroid of each cluster is recalculated to the average of the points in that cluster. 
4. Repeat 2-3 until points no longer get reassigned.


<img src="../images/kmeans.gif" alt="K-Means Illustration" height=600 width=600>

The downside of k-means is that it requires you to define in advance how many clusters there are expected to be in the data.

In [ ]:
from sklearn.datasets import make_blobs

blobs, blobs_labels = make_blobs(n_samples=300, 
                                 centers=4,
                                 cluster_std=0.60, 
                                 random_state=0)
plt.scatter(blobs[:, 0], blobs[:, 1], s=50);

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(blobs)

blobs_kmeans = kmeans.predict(blobs)

In [ ]:
plt.scatter(blobs[:, 0], blobs[:, 1], 
            c=blobs_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

### Gaussian Mixture Models

K-means is a partional clustering technique that uses a distance-based model. However, a major drawback of k-means is that these cluster models must be circular: k-means has no way of accounting for differently shaped clusters, such as oblong or elliptical. 

Let's examine some data like that: 

In [ ]:
data = pd.read_csv('../data/clustering_gmm.csv')
data.head()

In [ ]:
data = np.array([data['Weight'], data['Height']]).transpose()
data.shape

plt.figure(figsize=(7,7))
plt.scatter(data[:, 0], data[:, 1])

Although there are 4 clearly visible clusters, k-means is unable to correctly identify these. 

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(data)

data_kmeans = kmeans.predict(data)

In [ ]:
plt.scatter(data[:, 0], data[:, 1], 
            c=data_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

This is where **Gaussian Mixture Models** come in. Instead of a distanced-based model, GMM uses a distribution-based model. Gaussian Mixture Models are probabilistic models and use a soft clustering approach for distributing the points in different clusters. 

In the simplest case, this means GMMs can be used for finding clusters in the same manner as k-means. 

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4, init_params='kmeans')
gmm.fit(blobs)

blobs_gmm = gmm.predict(blobs)
plt.scatter(blobs[:, 0], blobs[:, 1],
            c=blobs_gmm, s=50, cmap='viridis')

However, as GMM contains a probabilistic model under the hood, it is also possible to find probabilistic cluster assignments.

In [ ]:
probabilities = gmm.predict_proba(blobs)
probabilities[0]

We can visualise this uncertainty by making the size of each point proportional to the certainty of its prediction. 

In [ ]:
size = 50 * probabilities.max(1) ** 2  # square emphasizes differences
plt.scatter(blobs[:, 0], blobs[:, 1], c=blobs_gmm, cmap='viridis', s=size);

K-means can be seen as a version of Gaussian Mixture Models that hard assigns the clusters. However, GMMs are - as they are distribution based - more applicable to non-circular data shapes. 

The algorithm for Gaussian Mixture Models is as follows: 
1. A number (k) of centroids are initialised. These are the centers of our clusters. 
2. For each point in the dataset, find the probability of membership in each cluster. 
3. For each cluster, update its location, normalisation, and shape based on all the data points weighted by their probability of membership to the cluster. 
4. Repeat 3-4 until points no longer get reassigned. 

![](../images/gmm.gif)

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=4)
gmm.fit(data)

data_gmm = gmm.predict(data)

In [ ]:
plt.scatter(data[:, 0], data[:, 1],
            c=data_gmm, s=50, cmap='viridis')

In [ ]:
help(KMeans)

# Density-based clustering


K-means only performs well when the what we want to take into account is the distance between the points and the centroid, _not_ when we're interested in the distances between the data points themselves. This is where density-based methods come in.

In [ ]:
from sklearn.datasets import make_moons

moons, moons_labels = make_moons(200, noise=.05, random_state=0)
plt.scatter(moons[:, 0], moons[:, 1]);

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(moons)

moons_kmeans = kmeans.predict(moons)

plt.scatter(moons[:, 0], moons[:, 1], 
            c=moons_kmeans,
            s=50, cmap='viridis');

### DBSCAN

DBSCAN stands for *Density-Based Spatial Clustering of Applications with Noise*. Essentially, it looks for areas of high density and assigns clusters to them, whereas points in less dense regions are not even included in the clusters and labeled as anomalies. It has two key settings:

- *eps*: maximum distance between two points to consider them as neighbors. If this distance is too large we might end up with all the points in one huge cluster, however, if it is too small we might not even form a single cluster.
- *min_points*: minimum number of points to form a cluster. If we set a low value for this parameters we might end up with a lot of really small clusters. However, a large value can stop the algorithm from creating any clusters at all.


<img src="../images/DBSCAN_search.gif" alt="DBSCAN Illustration" height=600 width=600>

DBSCAN looks at how many neighbors (closer than *eps*) each point has, considering neighbors all the points closer than a certain distance (*eps*). If more than *min_points* are neighbors, then a cluster is created, and this cluster is expanded with all the neighbors of the neighbors. Intuitively, it is important to have the input data scaled.

Again, scikit-learn has a neat implementation of this algorithm, which we can implement with relatively little effort: 

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=.2, min_samples=5)
moons_dbscan = dbscan.fit_predict(moons)

plt.scatter(moons[:, 0], moons[:, 1], 
            c=moons_dbscan,
            s=50, cmap='viridis');

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=10)

blobs_dbscan = dbscan.fit_predict(blobs)

plt.scatter(blobs[:, 0], blobs[:, 1], c=blobs_dbscan, s=50, cmap='viridis');

In [ ]:
blobs_dbscan

DBSCAN in this case can function as some form of outlier detection - after all, the points not assigned to one of the clusters. 

 A downside of DBSCAN, however, is that it does not have a `.predict` method, even though it does have a `.fit_predict()`

In [ ]:
# dbscan.predict(moons)

## Density-based Hierarchical Clustering 

Our data is, unfortunately, not always as neat as the blobs or moons dataset. Imagine the following dataset, a combination of moons, blobs and random data. 

In [ ]:
# Create data.
moons, _ = make_moons(n_samples=50, noise=0.05)
blobs, _ = make_blobs(n_samples=50, centers=[(-0.75,2.25), (1.0, 2.0)], cluster_std=0.25)
random = np.random.uniform(-2, 2, (20, 2))
data = np.vstack([moons, blobs, random])

# Plot data. 
plot_kwds = {'alpha' : 0.5, 's' : 20, 'linewidths':0}
plt.scatter(data[:, 0], data[:, 1], color='b', **plot_kwds)

### HBSCAN

HDBSCAN, the abbreviation for Hierarchical Density-Based Spatial Clustering of Applications with Noise, is an extension of DBSCAN and and appropriate choice for this type of data. DBSCAN is performed over varying epsilon values and the result is integrated, which allows HBSCAN to find clusters of varying densities - unlike DBSCAN. 

HDBSCAN is currently not available in scikit-learn, so we need to import it from a separate package `hdbscan`. 

In [ ]:
from hdbscan import HDBSCAN

hdbscan = HDBSCAN(min_cluster_size=5, gen_min_span_tree=True)
hdbscan.fit(data)

In [ ]:
plt.figure(figsize=(12, 8))
hdbscan.minimum_spanning_tree_.plot(edge_cmap='viridis', 
                                      edge_alpha=0.6, node_size=80, edge_linewidth=2);

This minimum spanning tree show the mutual reachability distance. 

$$d_{\text { mreach }-k}(a, b)=\max \left\{\operatorname{core}_{k}(a), \operatorname{core}_{k}(b), d(a, b)\right\}$$

This distance is a bit fancy. The $d(a,b)$ is just normal distance but $\operatorname{core}_{k}(a)$ is the distance the point would need to be eligable to be in a cluster. This means that outlier points immediately get a high distance. The algorithm therefore has a dendogram tree but the weights of the arcs are built by using information about the density of the points. This is what the dendogram would look like;

In [ ]:
plt.figure(figsize=(12, 8))
hdbscan.single_linkage_tree_.plot(cmap='viridis', colorbar=True);

Now here's a fun game to play. Given this tree we can use a notion of a **minimum cluster size** to select clusters. If at a split we see that the resulting cluster does not meet the requirement we will throw away the new cluster and let the parent keep it. If we do that then we are able to draw a new tree with new properties; 

- the thickness of the line denotes the number of points
- the length of the line denotes the $\lambda = 1/\text{distance}$ value of the points 
- each split is a split that is allowed by the minimum cluster rule 


In [ ]:
plt.figure(figsize=(8,8))
hdbscan.condensed_tree_.plot();

The algorithm now selects the clusters by optimising for the area of ink. Note that when a parent is selected to be a cluster that a child cannot be selected to be a cluster. 

This selection is done here; 

In [ ]:
plt.figure(figsize=(8,8))
hdbscan.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette());

Any point not in a selected cluster is considered to be a noisy point. We can see the result of this below (note corresponding colors).

In [ ]:
palette = sns.color_palette()

cluster_colors = [sns.desaturate(palette[col], sat) 
                  if col >= 0 else (0.5, 0.5, 0.5) for col, sat in 
                  zip(hdbscan.labels_, hdbscan.probabilities_)]

plt.scatter(data[:, 0], data[:, 1], c=cluster_colors, **plot_kwds)

A main downside of this clustering approach is that the algorithm does not offer `.predict()` method, much like DBSCAN.

# Hierarchical Clustering

HDBSCAN is a hierarchical, density-based clustering algorithm. There are, however, also pure hierarchical clustering algorithms. These algorithms aim to build a hierarchy of clusters and typically follow one of two approaches: 
- Agglomerative: a bottom-up approach where each observation starts in its own cluster and pairs of clusters are merged as one moves up the hierarchy
- Divisive: a top-down approach where all observations start in one cluster and splits are performed recursively as one moves down the hierarchy. 

![Example of agglomerative hierarchical clustering](../images/hierarchical.gif)

To show hierarchical clustering in practice, we're going to take a look at clients at a wholesale distributor based on their annual spending on diverse product categories, like milk, frozen, fresh, etc.

In [ ]:
df = pd.read_csv('../data/customers.csv')
df.head()

In [ ]:
from sklearn.preprocessing import normalize

data_scaled = normalize(df)

### Dendrogram

A dendrogram can be used to visualise the history of groupings and figure out the optimal number of clusters. 

First we create our dendogram using `scipy`. 

The linkage criteria refers to how the distance between clusters is calculated. A couple of options are: 
- Single linkage: $L(r, s) = min(D(x_{ri}, x_{sj}))$. The distance between two clusters is the shortest distance between two points in each cluster. 
- Complete linkage: $L(r, s) = max(D(x_{ri}, x_{sj}))$ The distance between two clusters is the longest distance between two points in each cluster.
- Average linkage: $L(r, s) = \frac{1}{n_r n_s} \sum^{n_r} \sum^{n_s} D(x_{ri}, x_{sj})$ The distance between clusters is the average distance between each point in one cluster to every point in other cluster.
- Ward linkage: The distance between clusters is the sum of squared differences within all clusters. 



In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

plt.figure(figsize=(10, 7))  

link = linkage(data_scaled, method='ward')
dend = dendrogram(link)

Once you have your dendrogram, the optimal number of cluster is found in the following way: 
1. Determine the largest vertical distance that does not intersect any of the other clusters. 
2. Draw a horizontal line through. 
3. The optimal number of clusters is equal to the number of vertical lines going through the horizontal line. 

In [ ]:
plt.figure(figsize=(10, 7))  

link = linkage(data_scaled, method='ward')
dend = dendrogram(link)

plt.axhline(y=6, color='r', linestyle='--')

In this case, the largest vertical distance is between approximately 5 and 12. The horizontal line drawn crosses 2 vertical lines, so the optimal number of clusters for this dataset is 2. 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=2, 
                                  affinity='euclidean', 
                                  linkage='ward')  
cluster.fit_predict(data_scaled)

Let's visualise our clusters on two variables: milk and grocery. 

In [ ]:
df_scaled = pd.DataFrame(data_scaled, columns=df.columns)
plt.scatter(df_scaled['Milk'], 
            df_scaled['Grocery'], 
            c=cluster.labels_);

# Conclusion

- The three families of clustering algorithms are: partional, density-based and hierarchical. 
- Partitional algorithms include k-means and Gaussian Mixture Models, where k-means is hard assigned version of GMMs and GMMs are able to provide probabilities.
- Density-based approaches include DBSCAN and HDBSCAN
- Hierarchical clustering methods are either agglomerative or divisive. These may correspond to meaningful taxonomies and a dendrogram can help find the most meaningful number of clusters.